In [ ]:
import json
import os

from langchain.schema import Document
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


def process_pdfs(chunk, folder_path):
    pdf_docs = []
    print(f'processing chunk: of len:{len(chunk)}')
    for file_name in chunk:
        if file_name.endswith('.pdf'):
            try:
                # TODO usar tesseract para extrair texto de pdfs imagem.
                loader = PDFPlumberLoader(os.path.join(folder_path, file_name))
                docs = loader.load()
                if len(docs) == 0:
                    print(f'warning: doc: {file_name} is empty')
                pdf_docs.extend(docs)
            except Exception as e:
                print(f'erro: {e} ao processar pdf {file_name}')
    return pdf_docs


def save_docs_to_jsonl(array, file_path: str) -> None:
    with open(file_path, 'w') as jsonl_file:
        for doc in array:
            jsonl_file.write(doc.json() + '\n')


def ensure_utf8(text):
    return text.encode('utf-8', errors='replace').decode('utf-8')


def load_docs_from_jsonl(file_path):
    array = []

    with open(file_path, 'r', encoding='utf-8') as jsonl_file:
        lines = jsonl_file.readlines()

    for i, line in enumerate(lines):
        # Converte a linha para UTF-8, garantindo que todos os caracteres sejam válidos
        utf8_line = ensure_utf8(line)
        data = json.loads(utf8_line)
        results = Document(**data)
        array.append(results)

    return array


# Carregar os documentos e aplicar a conversão
docs = load_docs_from_jsonl('data_finish.jsonl')

text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=200)
split_docs = text_splitter.split_documents(docs)
len(split_docs)

In [ ]:
import weaviate
from langchain_weaviate.vectorstores import WeaviateVectorStore

weaviate_client = weaviate.connect_to_local()
embeddings = OllamaEmbeddings(model="llama3", )
db = WeaviateVectorStore.from_documents(docs, embeddings, client=weaviate_client, index_name='protein_articles')


In [ ]:
from weaviate.classes.config import Configure, Property, DataType

weaviate_client = weaviate.connect_to_local()
weaviate_client.collections.create(
    "ProteinCollection",
    vectorizer_config=Configure.Vectorizer.text2vec_ollama(
        api_endpoint="http://ollama:11434",  # If using Docker, use this to contact your local Ollama instance
        model="mxbai-embed-large",  # The model to use, e.g. "nomic-embed-text"
    ),
    properties=[  # properties configuration is optional
        Property(name="title", data_type=DataType.TEXT, skip_vectorization=True),
        Property(name="body", data_type=DataType.TEXT),
        Property(name="page", data_type=DataType.INT, skip_vectorization=True),
        Property(name="doi", data_type=DataType.TEXT, skip_vectorization=True),
        Property(name="pk", data_type=DataType.TEXT, skip_vectorization=True),
        Property(name="proteins_structures", data_type=DataType.TEXT_ARRAY),
    ]
)
weaviate_client.close()

In [ ]:
weaviate_client = weaviate.connect_to_local()
collection = weaviate_client.collections.get("ProteinCollection")

with collection.batch.dynamic() as batch:
    for src_obj in split_docs:
        try:
            weaviate_obj = {
                "pk": src_obj.metadata.get("id", ''),
                "doi": src_obj.metadata.get("doi", ''),
                "page": src_obj.metadata.get("page", ''),
                "title": src_obj.metadata.get("title", ''),
                "body": src_obj.page_content,
                "proteins_structures": src_obj.metadata.get("proteins_structures", []),
            }

            batch.add_object(
                properties=weaviate_obj,
            )
        except Exception as err:
            print(f'Error: {err}, on id {src_obj.metadata.get("id", "")}')
weaviate_client.close()

In [ ]:
from weaviate.collections.classes.grpc import MetadataQuery

weaviate_client = weaviate.connect_to_local()
collection = weaviate_client.collections.get("ProteinCollection")

response = collection.query.hybrid(
    query="what is mobile loop ?",
    return_metadata=MetadataQuery(score=True, explain_score=True, distance=True, is_consistent=True),
    limit=3,
    # include_vector=True,
)

for o in response.objects:
    print(o.properties['body'][:100], '...')
    print(
        f'score: {o.metadata.score}, explain: {o.metadata.explain_score}, distance: {o.metadata.distance}, is_consistent: {o.metadata.is_consistent}')
    # print(o.vector["default"])
weaviate_client.close()

In [ ]:
weaviate_client = weaviate.connect_to_local()
collection = weaviate_client.collections.get("ProteinCollection")

count = 0
for item in collection.iterator():
    print(item.uuid, item.properties)
    count += 1
    if count > 3:
        break
weaviate_client.close()

In [136]:
from langchain_weaviate.vectorstores import WeaviateVectorStore

weaviate_client = weaviate.connect_to_local()
db = WeaviateVectorStore(client=weaviate_client, index_name='ProteinCollection',
                         embedding=OllamaEmbeddings(model="mxbai-embed-large", ), text_key='body')
response = db.similarity_search("what is mobile loop ?", alpha=0.5, k=20)
weaviate_client.close()
len(response)

/tmp/ipykernel_166673/3749480761.py:4: ResourceWarning: unclosed <socket.socket fd=83, family=2, type=1, proto=6, laddr=('127.0.0.1', 56954), raddr=('127.0.0.1', 11434)>
  db = WeaviateVectorStore(client=weaviate_client, index_name='ProteinCollection',


20

In [ ]:
# weaviate_client.connect()
# weaviate_client.collections.delete_all()
# weaviate_client.close()

In [ ]:
weaviate_client.connect()
try:
    collections = weaviate_client.collections.list_all()
finally:
    weaviate_client.close()
collections

In [137]:
weaviate_client = weaviate.connect_to_local()
db = WeaviateVectorStore(client=weaviate_client, index_name='ProteinCollection',
                         embedding=OllamaEmbeddings(model="mxbai-embed-large"),
                         text_key='body')
retriever = db.as_retriever(search_kwargs={'k': 20})

len(retriever.invoke('what is a mobile loop?'))


/tmp/ipykernel_166673/2645895327.py:2: ResourceWarning: unclosed <socket.socket fd=83, family=2, type=1, proto=6, laddr=('127.0.0.1', 48924), raddr=('127.0.0.1', 11434)>
  db = WeaviateVectorStore(client=weaviate_client, index_name='ProteinCollection',
/tmp/ipykernel_166673/2645895327.py:5: ResourceWarning: unclosed <socket.socket fd=80, family=2, type=1, proto=6, laddr=('127.0.0.1', 49608), raddr=('127.0.0.1', 11434)>
  retriever = db.as_retriever(search_kwargs = {'k': 20})
/mnt/TCC/protein-chat-flask/.venv/lib/python3.11/site-packages/weaviate/warnings.py:305: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


20

In [140]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama

prompt = """
    1. Use the following pieces of context to answer the question at the end.
            2. If you don't know the answer, just say "I don't know" but don't make up an answer on your own.
            3. Keep the answer crisp and limited to 3-4 sentences.
            
            Context: {context}
            
            Question: {question}
            
            After the answer, always say the source and page.
            Helpful Answer:
    """
llm = Ollama(model="llama3")
qa_chain_prompt = PromptTemplate.from_template(prompt)
rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | qa_chain_prompt
        | llm
        | StrOutputParser()
)

rag_chain.invoke('what is a mobile loop?')

'Based on the provided context, a mobile loop refers to switch loop I (blue and magenta) in the structure of human Pim-1 kinase. This mobile loop can adopt different conformations depending on the binding of nucleotides or other molecules.\n\nSource: Science Magazine, Page 1236086'